[old version of findmax(f, domain)](
https://github.com/JuliaLang/julia/pull/35316/files#diff-97ad8b900c30b39398c32d78754c404f4a6df9a153d45284b2c11ab54837deb9R797)

```julia
findmax(f, domain) = mapfoldl(x -> (f(x), x), _rf_findmax, domain)
 _rf_findmax((fm, m), (fx, x)) = isless(fm, fx) ? (fx, x) : (fm, m)
```

[current version](https://github.com/JuliaLang/julia/pull/41076/files#diff-97ad8b900c30b39398c32d78754c404f4a6df9a153d45284b2c11ab54837deb9R803)

```julia
findmax(f, domain) = mapfoldl( ((k, v),) -> (f(v), k), _rf_findmax, pairs(domain) )
 _rf_findmax((fm, im), (fx, ix)) = isless(fm, fx) ? (fx, ix) : (fm, im)
```

Ref. [`findmax` and friends: confusing behaviour to be introduced in 1.7](https://discourse.julialang.org/t/findmax-and-friends-confusing-behaviour-to-be-introduced-in-1-7/61904)

In [1]:
using OffsetArrays

In [2]:
VERSION

v"1.7.0-beta2"

In [3]:
f(x, y) = cos(x)*sin(y) + 0.1(x - y)

X = range(-2, 2; length=401)
Y = range(-2, 2; length=401)
XtimesY = Iterators.product(X, Y)

Base.Iterators.ProductIterator{Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}}((-2.0:0.01:2.0, -2.0:0.01:2.0))

In [4]:
val, idx = findmax(XtimesY) do (x, y); f(x, y) end

LoadError: MethodError: no method matching keys(::Base.Iterators.ProductIterator{Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}})
[0mClosest candidates are:
[0m  keys([91m::IOContext[39m) at show.jl:345
[0m  keys([91m::Base.Generator[39m) at generator.jl:54
[0m  keys([91m::Tuple[39m) at tuple.jl:72
[0m  ...

`pairs` function used in `findmax(f, domain)` function requires `keys` method.

In [5]:
methods(keys)

# 20 methods for generic function "keys":
[1] keys(io::IOContext) in Base at show.jl:345
[2] keys(g::Base.Generator) in Base at generator.jl:54
[3] keys(t::Tuple) in Base at tuple.jl:72
[4] keys(t::Tuple, t2::Tuple...) in Base at tuple.jl:77
[5] keys(itr::Base.SkipMissing) in Base at missing.jl:264
[6] keys(::Number) in Base at number.jl:93
[7] keys(v::Base.Pairs) in Base.Iterators at iterators.jl:256
[8] keys(a::AbstractDict) in Base at abstractdict.jl:103
[9] keys(nt::NamedTuple{names}) where names in Base at namedtuple.jl:306
[10] keys(s::IndexStyle, A::AbstractArray, B::AbstractArray...) in Base at abstractarray.jl:353
[11] keys(a::Test.GenericArray) in Test at D:\Julia-1.7.0\share\julia\stdlib\v1.7\Test\src\Test.jl:1819
[12] keys(a::AbstractVector) in Base at abstractarray.jl:136
[13] keys(a::AbstractArray) in Base at abstractarray.jl:135
[14] keys(s::AbstractString) in Base at strings/basic.jl:571
[15] keys(io::IO) in Base at show.jl:346
[16] keys(cmd::Cmd) in Base at process.jl:643
[17] keys(m::RegexMatch) in Base at regex.jl:219
[18] keys(m::Pkg.Types.Manifest) in Pkg.Types at D:\Julia-1.7.0\share\julia\stdlib\v1.7\Pkg\src\Types.jl:279
[19] keys(v::Core.SimpleVector) in Base at essentials.jl:609
[20] keys(r::Pkg.Registry.RegistryInstance) in Pkg.Registry at D:\Julia-1.7.0\share\julia\stdlib\v1.7\Pkg\src\Registry\registry_instance.jl:386

In [6]:
keys(collect(XtimesY))

401×401 CartesianIndices{2, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}}:
 CartesianIndex(1, 1)    CartesianIndex(1, 2)    …  CartesianIndex(1, 401)
 CartesianIndex(2, 1)    CartesianIndex(2, 2)       CartesianIndex(2, 401)
 CartesianIndex(3, 1)    CartesianIndex(3, 2)       CartesianIndex(3, 401)
 CartesianIndex(4, 1)    CartesianIndex(4, 2)       CartesianIndex(4, 401)
 CartesianIndex(5, 1)    CartesianIndex(5, 2)       CartesianIndex(5, 401)
 CartesianIndex(6, 1)    CartesianIndex(6, 2)    …  CartesianIndex(6, 401)
 CartesianIndex(7, 1)    CartesianIndex(7, 2)       CartesianIndex(7, 401)
 CartesianIndex(8, 1)    CartesianIndex(8, 2)       CartesianIndex(8, 401)
 CartesianIndex(9, 1)    CartesianIndex(9, 2)       CartesianIndex(9, 401)
 CartesianIndex(10, 1)   CartesianIndex(10, 2)      CartesianIndex(10, 401)
 CartesianIndex(11, 1)   CartesianIndex(11, 2)   …  CartesianIndex(11, 401)
 CartesianIndex(12, 1)   CartesianIndex(12, 2)      CartesianIndex(12, 401)
 CartesianIndex(13, 1) 

In [7]:
val, idx = findmax(collect(XtimesY)) do (x, y); f(x, y) end

(0.8529538721652244, CartesianIndex(211, 348))

But `collect` causes memory allocations and I do not feel like it.

In [8]:
axes(XtimesY)

(Base.OneTo(401), Base.OneTo(401))

In [9]:
CartesianIndices(axes(XtimesY))

401×401 CartesianIndices{2, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}}:
 CartesianIndex(1, 1)    CartesianIndex(1, 2)    …  CartesianIndex(1, 401)
 CartesianIndex(2, 1)    CartesianIndex(2, 2)       CartesianIndex(2, 401)
 CartesianIndex(3, 1)    CartesianIndex(3, 2)       CartesianIndex(3, 401)
 CartesianIndex(4, 1)    CartesianIndex(4, 2)       CartesianIndex(4, 401)
 CartesianIndex(5, 1)    CartesianIndex(5, 2)       CartesianIndex(5, 401)
 CartesianIndex(6, 1)    CartesianIndex(6, 2)    …  CartesianIndex(6, 401)
 CartesianIndex(7, 1)    CartesianIndex(7, 2)       CartesianIndex(7, 401)
 CartesianIndex(8, 1)    CartesianIndex(8, 2)       CartesianIndex(8, 401)
 CartesianIndex(9, 1)    CartesianIndex(9, 2)       CartesianIndex(9, 401)
 CartesianIndex(10, 1)   CartesianIndex(10, 2)      CartesianIndex(10, 401)
 CartesianIndex(11, 1)   CartesianIndex(11, 2)   …  CartesianIndex(11, 401)
 CartesianIndex(12, 1)   CartesianIndex(12, 2)      CartesianIndex(12, 401)
 CartesianIndex(13, 1) 

In [10]:
Base.keys(pr::Iterators.ProductIterator) = CartesianIndices(axes(pr))
F((x, y)) = f(x, y)

@show val, idx = findmax(F, XtimesY)
@show f(X[idx[1]], Y[idx[2]])
@show argmax(F.(XtimesY));

(val, idx) = findmax(F, XtimesY) = (0.8529538721652244, CartesianIndex(211, 348))
f(X[idx[1]], Y[idx[2]]) = 0.8529538721652244
argmax(F.(XtimesY)) = CartesianIndex(211, 348)


In [11]:
#valargmax(f, X) = (x = argmax(f, X); (f(x), x))
struct ValArg{F} <:Function f::F end
(valarg::ValArg)(x) = (valarg.f(x), x)
valargmax(f, X) = mapfoldl(ValArg(f), Base._rf_findmax, X)
valargmax(X) = valargmax(Base.Fix1(getindex, X), keys(X))
F((x, y)) = f(x, y)

@show val, idx = findmax(F, XtimesY)
@show val, (X[idx[1]], Y[idx[2]])
@show argmax(F, XtimesY)
@show valargmax(F, XtimesY)
@show valargmax(F.(XtimesY))
@show findmax(F, XtimesY);

(val, idx) = findmax(F, XtimesY) = (0.8529538721652244, CartesianIndex(211, 348))
(val, (X[idx[1]], Y[idx[2]])) = (0.8529538721652244, (0.1, 1.47))
argmax(F, XtimesY) = (0.1, 1.47)
valargmax(F, XtimesY) = (0.8529538721652244, (0.1, 1.47))
valargmax(F.(XtimesY)) = (0.8529538721652244, CartesianIndex(211, 348))
findmax(F, XtimesY) = (0.8529538721652244, CartesianIndex(211, 348))


In [12]:
Base.keys(rv::Iterators.Reverse) = keys(reverse(rv.itr))

@show val, idx = findmax(sin, Iterators.Reverse(X))
@show sin(reverse(X)[idx]);

(val, idx) = findmax(sin, Iterators.Reverse(X)) = (0.9999996829318346, 44)
sin((reverse(X))[idx]) = 0.9999996829318346


In [13]:
Base.keys(en::Iterators.Enumerate) = keys(en.itr)
G((i, x)) = sin(x)

@show val, idx = findmax(G, enumerate(X))
@show sin(X[idx]);

(val, idx) = findmax(G, enumerate(X)) = (0.9999996829318346, 358)
sin(X[idx]) = 0.9999996829318346


In [14]:
Base.keys(zp::Iterators.Zip) = Base.OneTo(length(zp))
H((x, y)) = cos(x) * sin(y)

@show val, idx = findmax(H, zip(X, reverse(Y)))
@show cos(X[idx]) * sin(reverse(Y)[idx]);

(val, idx) = findmax(H, zip(X, reverse(Y))) = (0.49997882324937, 122)
cos(X[idx]) * sin((reverse(Y))[idx]) = 0.49997882324937


In [15]:
Base.keys(ac::Iterators.Accumulate) = keys(ac.itr)
A = OffsetArray(range(-2, 2; length=401), -200:200)

@show val, idx = findmax(sin, Iterators.accumulate(+, A; init=π))
@show sin(sum(A[begin:idx]) + π);

(val, idx) = findmax(sin, Iterators.accumulate(+, A; init = π)) = (0.9999998945903477, -31)
sin(sum(A[begin:idx]) + π) = 0.9999998945903477


In [16]:
Base.keys(tk::Iterators.Take) = Base.OneTo(length(tk))

@show val, idx = findmax(sin, Iterators.take(reverse(X), 100))
@show sin(reverse(X)[idx]);

(val, idx) = findmax(sin, Iterators.take(reverse(X), 100)) = (0.9999996829318346, 44)
sin((reverse(X))[idx]) = 0.9999996829318346


In [17]:
methods(keys, Main)

# 6 methods for generic function "keys":
[1] keys(pr::Base.Iterators.ProductIterator) in Main at In[10]:1
[2] keys(en::Base.Iterators.Enumerate) in Main at In[13]:1
[3] keys(rv::Base.Iterators.Reverse) in Main at In[12]:1
[4] keys(tk::Base.Iterators.Take) in Main at In[16]:1
[5] keys(ac::Base.Iterators.Accumulate) in Main at In[15]:1
[6] keys(zp::Base.Iterators.Zip) in Main at In[14]:1

In [18]:
?keys

search: keys keytype KeyError haskey getkey UndefKeywordError WeakKeyDict



```
keys(a::AbstractArray)
```

Return an efficient array describing all valid indices for `a` arranged in the shape of `a` itself.

They keys of 1-dimensional arrays (vectors) are integers, whereas all other N-dimensional arrays use [`CartesianIndex`](@ref) to describe their locations.  Often the special array types [`LinearIndices`](@ref) and [`CartesianIndices`](@ref) are used to efficiently represent these arrays of integers and `CartesianIndex`es, respectively.

Note that the `keys` of an array might not be the most efficient index type; for maximum performance use  [`eachindex`](@ref) instead.

---

```
keys(iterator)
```

For an iterator or collection that has keys and values (e.g. arrays and dictionaries), return an iterator over the keys.

---

```
keys(a::AbstractDict)
```

Return an iterator over all keys in a dictionary. `collect(keys(a))` returns an array of keys. When the keys are stored internally in a hash table, as is the case for `Dict`, the order in which they are returned may vary. But `keys(a)` and `values(a)` both iterate `a` and return the elements in the same order.

# Examples

```jldoctest
julia> D = Dict('a'=>2, 'b'=>3)
Dict{Char, Int64} with 2 entries:
  'a' => 2
  'b' => 3

julia> collect(keys(D))
2-element Vector{Char}:
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'b': ASCII/Unicode U+0062 (category Ll: Letter, lowercase)
```

---

```
keys(m::RegexMatch) -> Vector
```

Return a vector of keys for all capture groups of the underlying regex. A key is included even if the capture group fails to match. That is, `idx` will be in the return value even if `m[idx] == nothing`.

Unnamed capture groups will have integer keys corresponding to their index. Named capture groups will have string keys.

!!! compat "Julia 1.6"
    This method was added in Julia 1.6


# Examples

```jldoctest
julia> keys(match(r"(?<hour>\d+):(?<minute>\d+)(am|pm)?", "11:30"))
3-element Vector{Any}:
  "hour"
  "minute"
 3
```


https://github.com/JuliaLang/julia/blob/16f433bb13cfc87eea21d26a797dac7b34a41d86/base/abstractdict.jl#L71

In [19]:
?Base.isgreater

```
isgreater(x, y)
```

Not the inverse of `isless`! Test whether `x` is greater than `y`, according to a fixed total order compatible with `min`.

Defined with `isless`, this function is usually `isless(y, x)`, but `NaN` and [`missing`](@ref) are ordered as smaller than any ordinary value with `missing` smaller than `NaN`.

So `isless` defines an ascending total order with `NaN` and `missing` as the largest values and `isgreater` defines a descending total order with `NaN` and `missing` as the smallest values.

!!! note
    Like `min`, `isgreater` orders containers (tuples, vectors, etc) lexicographically with `isless(y, x)` rather than recursively with itself:

    ```jldoctest
    julia> Base.isgreater(1, NaN) # 1 is greater than NaN
    true

    julia> Base.isgreater((1,), (NaN,)) # But (1,) is not greater than (NaN,)
    false

    julia> sort([1, 2, 3, NaN]; lt=Base.isgreater)
    4-element Vector{Float64}:
       3.0
       2.0
       1.0
     NaN

    julia> sort(tuple.([1, 2, 3, NaN]); lt=Base.isgreater)
    4-element Vector{Tuple{Float64}}:
     (NaN,)
     (3.0,)
     (2.0,)
     (1.0,)
    ```


# Implementation

This is unexported. Types should not usually implement this function. Instead, implement `isless`.


In [20]:
?findmax

search: findmax findmax! findmin findmin!



```
findmax(f, domain) -> (f(x), index)
```

Returns a pair of a value in the codomain (outputs of `f`) and the index of the corresponding value in the `domain` (inputs to `f`) such that `f(x)` is maximised. If there are multiple maximal points, then the first one will be returned.

`domain` must be a non-empty iterable.

Values are compared with `isless`.

!!! compat "Julia 1.7"
    This method requires Julia 1.7 or later.


# Examples

```jldoctest
julia> findmax(identity, 5:9)
(9, 5)

julia> findmax(-, 1:10)
(-1, 1)

julia> findmax(first, [(1, :a), (3, :b), (3, :c)])
(3, 2)

julia> findmax(cos, 0:π/2:2π)
(1.0, 1)
```

---

```
findmax(itr) -> (x, index)
```

Return the maximal element of the collection `itr` and its index or key. If there are multiple maximal elements, then the first one will be returned. Values are compared with `isless`.

See also: [`findmin`](@ref), [`argmax`](@ref), [`maximum`](@ref).

# Examples

```jldoctest
julia> findmax([8, 0.1, -9, pi])
(8.0, 1)

julia> findmax([1, 7, 7, 6])
(7, 2)

julia> findmax([1, 7, 7, NaN])
(NaN, 4)
```

---

```
findmax(A; dims) -> (maxval, index)
```

For an array input, returns the value and index of the maximum over the given dimensions. `NaN` is treated as greater than all other values except `missing`.

# Examples

```jldoctest
julia> A = [1.0 2; 3 4]
2×2 Matrix{Float64}:
 1.0  2.0
 3.0  4.0

julia> findmax(A, dims=1)
([3.0 4.0], CartesianIndex{2}[CartesianIndex(2, 1) CartesianIndex(2, 2)])

julia> findmax(A, dims=2)
([2.0; 4.0;;], CartesianIndex{2}[CartesianIndex(1, 2); CartesianIndex(2, 2);;])
```


In [21]:
findmax([8, 0.1, -9, pi])

(8.0, 1)

In [22]:
A = [1.0 2; 3 4]
findmax(A, dims=1)

([3.0 4.0], CartesianIndex{2}[CartesianIndex(2, 1) CartesianIndex(2, 2)])

In [23]:
methods(findmax)

# 6 methods for generic function "findmax":
[1] findmax(a::BitArray) in Base at bitarray.jl:1561
[2] findmax(A::SparseArrays.AbstractSparseMatrixCSC) in SparseArrays at D:\Julia-1.7.0\share\julia\stdlib\v1.7\SparseArrays\src\sparsematrix.jl:2145
[3] findmax(A::SparseArrays.AbstractSparseMatrixCSC{Tv, Ti}, region) where {Tv, Ti} in SparseArrays at D:\Julia-1.7.0\share\julia\stdlib\v1.7\SparseArrays\src\sparsematrix.jl:2143
[4] findmax(A::AbstractArray; dims) in Base at reducedim.jl:1052
[5] findmax(itr) in Base at reduce.jl:828
[6] findmax(f, domain) in Base at reduce.jl:803

In [24]:
findmax(-(-4:5).^2)

(0, 5)

In [25]:
findmax(x -> -x^2, -4:5)

(0, 5)

In [26]:
valindargmax(f, X) = valargmax(f∘last, pairs(X))

X = range(-2, 2; length=401)
@show m, (i, x) = valindargmax(sin, X)
m, (i, x)

(m, (i, x)) = valindargmax(sin, X) = (0.9999996829318346, 358 => 1.57)


(0.9999996829318346, (358, 1.57))

In [27]:
F((x, y)) = f(x, y)

X = range(-2, 2; length=401)
Y = range(-2, 2; length=401)
XtimesY = Iterators.product(X, Y)
@show m, (i, x) = valindargmax(F, XtimesY)
m, (i, x)

(m, (i, x)) = valindargmax(F, XtimesY) = (0.8529538721652244, CartesianIndex(211, 348) => (0.1, 1.47))


(0.8529538721652244, (CartesianIndex(211, 348), (0.1, 1.47)))